In [1]:
import scipy.io as sio
import pandas as pd


In [19]:
class SignalFeatures():
    # signal basic parameters
    rest_original_signal = []   # 1 col signal
    active_original_signal = [] # 1 col signal
    fs = 0

    rest_signal_len = 0
    active_signal_len = 0
    # data segment list, 2D list
    # initiate in signal_segment function
    rest_signal_segment = []
    active_signal_segment = []
    # init
    def __init__(self, rest_signal, active_signal, sample_freq):
        # input signal: 1D list []
        self.rest_original_signal = rest_signal
        self.active_original_signal = active_signal
        fs = sample_freq

        self.rest_signal_len = len(rest_signal)
        self.active_signal_len = len(active_signal)
        pass

    def signal_segment(self, window_len, step_len):
        # split rest signal
        if self.rest_signal_len < window_len:
            print("Rest signal length is below the window length")
            self.rest_signal_segment.append(self.rest_original_signal)
        else:
            for i in range((self.rest_signal_len - window_len)//step_len):
                self.rest_signal_segment.append(self.rest_original_signal[i*step_len : i*step_len + window_len])
                pass
            pass

        # split active signal
        if self.active_signal_len < window_len:
            print("active signal length is below the window length")
            self.active_signal_segment.append(self.active_original_signal)
        else:
            for i in range((self.active_signal_len - window_len)//step_len):
                self.active_signal_segment.append(self.active_original_signal[i*step_len : i*step_len + window_len])
                pass
            pass

    pass


class FMGFeatures(SignalFeatures):

    pass


class sEMGFeatures(SignalFeatures):

    pass

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_19832/2649532475.py, line 1)

In [4]:
# 读取预实验路径下的几次实验数据
# file path format: 'D:\\folder path\\subject name\\label_(rest/active).mat'
file_folder_path = 'D:\code\data\iFEMG_data_set'
subjects = ['\zpk1', '\zpk2', '\zpk3', '\zpk4', '\zpk5', '\zpk6']
mat_name = ['d0', 'd2', 'd5', 'd7', 'dm']

signal_df = pd.DataFrame(columns = ('subject_name', 'rest_signal', 'active_signal', 'label', 'sensor_channel'))

for i in subjects:
    for j in mat_name:
        # print('processing: ', i + j)
        # build data frame, every row has a unique label
        signal_df = signal_df.append({'subject_name' : i,
                                    'rest_signal': sio.loadmat(file_folder_path + i + '\\' + j + '_rest.mat')[j + '_rest'],
                                    'active_signal': sio.loadmat(file_folder_path + i + '\\' + j + '_active.mat')[j + '_active'], 
                                    'label' : j,
                                    'sensor_channel' : 'bicps_br'}, ignore_index=True)

    pass

In [14]:
# 使用boolean值索引出某一名受试者的实验数据
data = signal_df.loc[signal_df.loc[:, 'subject_name'] == '\zpk1']
print(data.head())
print(type(data.loc[0, 'rest_signal']))

  subject_name                                        rest_signal  \
0        \zpk1  [[88.0, 311.8869405619067], [88.0, 283.8395630...   
1        \zpk1  [[106.0, -95.733998252022], [106.0, -159.97312...   
2        \zpk1  [[0.0, -115.96887136551932], [0.0, -118.489493...   
3        \zpk1  [[115.0, -152.71052914404424], [115.0, -188.92...   
4        \zpk1  [[79.0, -122.4545388065741], [79.0, -73.193093...   

                                       active_signal label sensor_channel  
0  [[203.0, -295.63014277918353], [203.0, -287.21...    d0       bicps_br  
1  [[283.0, -118.90756361272028], [283.0, -653.43...    d2       bicps_br  
2  [[300.0, -92.69143911160802], [300.0, 925.2660...    d5       bicps_br  
3  [[265.0, -1168.724526078126], [265.0, -240.337...    d7       bicps_br  
4  [[265.0, 868.3033449275356], [265.0, 800.74755...    dm       bicps_br  
<class 'numpy.ndarray'>


In [18]:
for row in data.itertuples():
    print(type(row))

<class 'pandas.core.frame.Pandas'>
<class 'pandas.core.frame.Pandas'>
<class 'pandas.core.frame.Pandas'>
<class 'pandas.core.frame.Pandas'>
<class 'pandas.core.frame.Pandas'>
